In [360]:
import mido
import csv
import audiolazy

In [361]:
file_name = 'jesus.mid'

In [362]:
mid = mido.MidiFile(file_name)

In [428]:
active_notes = {}
all_notes = {}
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    tempo = 0
    time = 0
    all_notes[i] = []
    for msg in track:
        # print(msg)
        if msg.type == 'note_on':
            msg_dict = msg.dict()
            time += msg_dict['time']
            if msg_dict['velocity'] > 0:
                active_notes[msg_dict['note']] = time
            elif msg_dict['velocity'] == 0:
                all_notes[i].append({'note': msg_dict['note'], 'start': active_notes[msg_dict['note']], 'end': time})
                del active_notes[msg_dict['note']]
            # msg_dict['active'] = active_notes
            # print(msg_dict)
        if msg.is_meta:
            if msg.type == 'set_tempo':
                tempo = msg.dict()['tempo']
                # print(tempo)

Track 0: 
Track 1: 


Split into single note at a time tracks

In [429]:
streams = []
for channel_num,notes in all_notes.iteritems():
    while notes != []:
        stream = []
        current_end = 0
        for note in notes:
            if note['start'] >= current_end:
                stream.append(note)
                current_end = note['end']
        streams.append(stream)
        for note in stream:
            notes.remove(note)
num_turtles = len(streams)

Calculate cell length

In [430]:
all_notes = [item for sublist in streams for item in sublist]

In [431]:
all_notes

[{'end': 455, 'note': 67, 'start': 0},
 {'end': 935, 'note': 64, 'start': 480},
 {'end': 1415, 'note': 64, 'start': 960},
 {'end': 1895, 'note': 62, 'start': 1440},
 {'end': 2375, 'note': 64, 'start': 1920},
 {'end': 2855, 'note': 67, 'start': 2400},
 {'end': 3791, 'note': 67, 'start': 2880},
 {'end': 4295, 'note': 69, 'start': 3840},
 {'end': 4775, 'note': 69, 'start': 4320},
 {'end': 5255, 'note': 72, 'start': 4800},
 {'end': 5735, 'note': 69, 'start': 5280},
 {'end': 6215, 'note': 69, 'start': 5760},
 {'end': 6695, 'note': 67, 'start': 6240},
 {'end': 7631, 'note': 64, 'start': 6720},
 {'end': 8135, 'note': 67, 'start': 7680},
 {'end': 8615, 'note': 64, 'start': 8160},
 {'end': 9095, 'note': 64, 'start': 8640},
 {'end': 9575, 'note': 62, 'start': 9120},
 {'end': 10055, 'note': 64, 'start': 9600},
 {'end': 10535, 'note': 67, 'start': 10080},
 {'end': 11471, 'note': 67, 'start': 10560},
 {'end': 11975, 'note': 69, 'start': 11520},
 {'end': 12455, 'note': 69, 'start': 12000},
 {'end': 

In [432]:
differences = [(y['start']-x['start']) for x, y in zip(all_notes[:-1], all_notes[1:])]
mode_difference = max(set(differences), key=times.count)

In [433]:
lengths = [(x['end'] - x['start']) for x in [item for sublist in streams for item in sublist]]
mode_length = max(set(lengths), key=times.count)

In [434]:
mode_difference

480

In [435]:
mode_length

227

In [436]:
mode = mode_length

In [437]:
mode

227

In [438]:
ratio = (float(mode_difference) / mode_length)/(mode_difference / mode_length)

In [439]:
ratio_int = (mode_difference / mode_length)

In [440]:
for stream in streams:
    for note in stream:
        note['length'] = ((float(note['end']) - note['start'])/mode) 

In [441]:
streams

[[{'end': 455, 'length': 2.004405286343612, 'note': 67, 'start': 0},
  {'end': 935, 'length': 2.004405286343612, 'note': 64, 'start': 480},
  {'end': 1415, 'length': 2.004405286343612, 'note': 64, 'start': 960},
  {'end': 1895, 'length': 2.004405286343612, 'note': 62, 'start': 1440},
  {'end': 2375, 'length': 2.004405286343612, 'note': 64, 'start': 1920},
  {'end': 2855, 'length': 2.004405286343612, 'note': 67, 'start': 2400},
  {'end': 3791, 'length': 4.013215859030837, 'note': 67, 'start': 2880},
  {'end': 4295, 'length': 2.004405286343612, 'note': 69, 'start': 3840},
  {'end': 4775, 'length': 2.004405286343612, 'note': 69, 'start': 4320},
  {'end': 5255, 'length': 2.004405286343612, 'note': 72, 'start': 4800},
  {'end': 5735, 'length': 2.004405286343612, 'note': 69, 'start': 5280},
  {'end': 6215, 'length': 2.004405286343612, 'note': 69, 'start': 5760},
  {'end': 6695, 'length': 2.004405286343612, 'note': 67, 'start': 6240},
  {'end': 7631, 'length': 4.013215859030837, 'note': 64, '

In [442]:
rounding_base = 0.1
for stream in streams:
    for note in stream:
        note['length'] = rounding_base * round(note['length']/rounding_base)
        note['start'] = round(rounding_base * round((float(note['start'])/mode_difference*ratio_int)/rounding_base))
        # note['end'] = rounding_base * round((note['end']/mode)/rounding_base)
        note['end'] = note['start'] + note['length']

In [443]:
streams

[[{'end': 2.0, 'length': 2.0, 'note': 67, 'start': 0.0},
  {'end': 4.0, 'length': 2.0, 'note': 64, 'start': 2.0},
  {'end': 6.0, 'length': 2.0, 'note': 64, 'start': 4.0},
  {'end': 8.0, 'length': 2.0, 'note': 62, 'start': 6.0},
  {'end': 10.0, 'length': 2.0, 'note': 64, 'start': 8.0},
  {'end': 12.0, 'length': 2.0, 'note': 67, 'start': 10.0},
  {'end': 16.0, 'length': 4.0, 'note': 67, 'start': 12.0},
  {'end': 18.0, 'length': 2.0, 'note': 69, 'start': 16.0},
  {'end': 20.0, 'length': 2.0, 'note': 69, 'start': 18.0},
  {'end': 22.0, 'length': 2.0, 'note': 72, 'start': 20.0},
  {'end': 24.0, 'length': 2.0, 'note': 69, 'start': 22.0},
  {'end': 26.0, 'length': 2.0, 'note': 69, 'start': 24.0},
  {'end': 28.0, 'length': 2.0, 'note': 67, 'start': 26.0},
  {'end': 32.0, 'length': 4.0, 'note': 64, 'start': 28.0},
  {'end': 34.0, 'length': 2.0, 'note': 67, 'start': 32.0},
  {'end': 36.0, 'length': 2.0, 'note': 64, 'start': 34.0},
  {'end': 38.0, 'length': 2.0, 'note': 64, 'start': 36.0},
  {'en

Create list for cells in that single turtle track

In [444]:
max_time = int(max([x['end'] for x in [item for sublist in streams for item in sublist]]))
start_cells = 'A2:A' + str(1+num_turtles)
instructions = 'r m' + str(max_time-1)
speed = 1
turtles = [['!turtle(' + start_cells + ', ' + instructions + ', ' + str(speed) + ', 1)']]
for stream in streams:
    cells = [""] * max_time
    for note in stream:
        start = int(note['start'])
        cells[start] = audiolazy.midi2str(note['note'])
        for rest_duration in range(1,int(note['length'])):
            cells[start+rest_duration] = '-'
    turtles.append(cells)

Combine lists into csv

In [445]:
csv_name = file_name.split('.')[0] + '.csv'
with open(csv_name, "wb") as f:
    writer = csv.writer(f)
    writer.writerows(turtles)

dynamics